In [0]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive

In [2]:
ls

datalab/  drive/


In [5]:
cd drive

/content/drive


In [0]:
import collections
import os
import sys

import tensorflow as tf
import numpy as np

from tensorflow.contrib.seq2seq import sequence_loss

import math
import re
import matplotlib.pyplot as plt

import pickle

!pip install -q mosestokenizer
from mosestokenizer import *

### Split the training set into small txt files

In [0]:
def split_files_en(big_filename, limit):
    file_count = 0
    url_list = []

    with open(big_filename) as f:
        for line in f:
            url_list.append(line)
            if len(url_list) < limit:
                continue
            file_name = "./small_txt/" + str(file_count) + "_en.txt"
        
            with open(file_name, 'w') as file:
                for url in url_list[:-1]:
                    file.write(url)
                file.write(url_list[-1].strip())
                url_list = []
                file_count += 1

    if url_list:
        file_name = str(file_count) + ".txt"
        with open(file_name, 'w') as file:
            for url in url_list:
                file.write(url)
    print("done")
    
    
def split_files_fr(big_filename, limit):
    file_count = 0
    url_list = []

    with open(big_filename) as f:
        for line in f:
            url_list.append(line)
            if len(url_list) < limit:
                continue
            file_name = "../small_txt/" + str(file_count) + "_fr.txt"
        
            with open(file_name, 'w') as file:
                for url in url_list[:-1]:
                    file.write(url)
                file.write(url_list[-1].strip())
                url_list = []
                file_count += 1

    if url_list:
        file_name = str(file_count) + ".txt"
        with open(file_name, 'w') as file:
            for url in url_list:
                file.write(url)
    print("done")

In [0]:
#### count the number of lines in the text

en_count = -1
for en_count,line in enumerate(open(r"./translation_data/europarl-v7.fr-en.en")):
    pass
en_count += 1
print(en_count)

fr_count = -1
for fr_count,line in enumerate(open(r"./translation_data/europarl-v7.fr-en.fr")):
    pass
fr_count += 1
print(fr_count)

2007723
2007723


In [0]:
limit = 150000
bigfilename_en = "./europarl-v7.fr-en.en"
#bigfilename_fr = "./europarl-v7.fr-en.fr"

split_files_en(bigfilename_en, limit)
#split_files_fr(bigfilename_fr, limit)

### Build vocab dictionary

In [0]:
def _read_words(filename):
  with tf.gfile.GFile(filename, "r") as f: 
    output = f.read().replace("\n", " eos ").replace(".", " .")
    output = re.sub('[0-9]+', 'N', output)
    return output

In [0]:
def train_vocab_en(n):  
    # get vocab in train which show more than n times
    
    vocab = {'<PAD>': 0, '<OOV>': 1}
    
    word_count = collections.defaultdict(float)
    
    en_tokenize = MosesTokenizer('en')
    
    for i in range(13):
      data = _read_words("./small_txt/" + str(i) + "_en.txt")
      data = en_tokenize(data)
    
      for token in data:
        word_count[token] += 1.0
                          
    total_word = set(word_count)
    
    for word in total_word:
        if word_count[word] > n:
            vocab[word] = len(vocab)  # word only show less than n times became oov
            
    return vocab
  
def train_vocab_fr(n):  
    # get vocab in train which show more than n times
    
    vocab = {'<PAD>': 0, '<OOV>': 1}
    
    word_count = collections.defaultdict(float)
    
    en_tokenize = MosesTokenizer('fr')
    
    for i in range(13):
      data = _read_words("./small_txt/" + str(i) + "_fr.txt")
      data = en_tokenize(data)
    
      for token in data:
        word_count[token] += 1.0
                          
    total_word = set(word_count)
    
    print(total_word)
    for word in total_word:
        if word_count[word] > n:
            vocab[word] = len(vocab)  # word only show less than n times became oov
            
    return vocab

In [0]:
vocab_en = train_vocab_en(5)  

In [10]:
len(vocab_en)

38841

In [0]:
vocab_fr = train_vocab_fr(10)  

In [0]:
## save dict to txt file

f = open("./dictionary/en_word_to_id_new.txt", "wb")
pickle.dump(vocab_en, f)
f.close()

In [0]:
## save dict to txt file

f = open("./en_word_to_id.txt", "wb")
pickle.dump(vocab_en, f)
f.close()

f = open("./fr_word_to_id.txt", "wb")
pickle.dump(vocab_fr, f)
f.close()

In [0]:
## read dict from txt file

f = open("./en_word_to_id.txt", "rb")
en_word_to_id = pickle.load(f)
f.close()

f = open("./fr_word_to_id.txt", "rb")
fr_word_to_id = pickle.load(f)
f.close()

In [0]:
en_vocab_size = len(en_word_to_id)

fr_word_to_id['<beg>'] = len(fr_word_to_id)
fr_vocab_size = len(fr_word_to_id)

print(en_vocab_size)
print(fr_vocab_size)

30772

### Word_to_id

In [0]:
def _file_to_word_ids(data, word_to_id):
  
  id_list = []
  
  for word in data:
    if word in word_to_id:
      id_list.append(word_to_id[word])
    else:
      id_list.append(1)
          
  return id_list

In [0]:
def preprocess_train_data(pre_data, word_to_id, max_length):
    pre_data_array = np.asarray(pre_data)
    last_start = 0
    data = []
    each_sen_len = []
    
    for i in range(len(pre_data_array)):
        if pre_data_array[i]==word_to_id['eos']:
            if max_length >= len(pre_data_array[last_start:(i+1)]):                
              data.append(pre_data_array[last_start:(i+1)])
              each_sen_len.append(i+1-last_start)
              
            else:
              shorten_sentences = pre_data_array[last_start:(last_start+max_length-1)]
              shorten_sentences = np.concatenate((shorten_sentences, np.asarray([word_to_id['eos']])), axis=0)
              data.append(shorten_sentences)
              each_sen_len.append(max_length) 
            
            last_start = i+1
            
    out_sentences = np.full([len(data), max_length], word_to_id['<PAD>'], dtype=np.int32)
    for i in range(len(data)):
        out_sentences[i,:len(data[i])] = data[i]    
    return out_sentences, np.asarray(each_sen_len)
            
    return data, each_sen_len

In [0]:
def generate_input_en(en_file, en_word_to_id, max_length):
  
    en_data = _read_words(en_file)

    en_tokenize = MosesTokenizer('en')

    en_data = en_tokenize(en_data)

    en_data_id = _file_to_word_ids(en_data, en_word_to_id)

    en_input, en_input_len = preprocess_train_data(en_data_id, en_word_to_id, max_length)
    
    return en_input, en_input_len
  
  
  
def generate_output_fr(fr_file, fr_word_to_id, max_length):
    
    fr_data = _read_words(fr_file)

    fr_tokenize = MosesTokenizer('fr')

    fr_data = fr_tokenize(fr_data)

    fr_data_id = _file_to_word_ids(fr_data, fr_word_to_id)

    fr_output, fr_output_len = preprocess_train_data(fr_data_id, fr_word_to_id,max_length=30)

    out_beg_token = fr_word_to_id['<beg>']*np.ones((fr_output.shape[0], 1), dtype=np.int32)

    fr_output = np.concatenate((out_beg_token, fr_output), axis=1)

    return fr_output,fr_output_len

In [0]:
def batch_producer(raw_data, raw_data_len, batch_size):    
    data_len = len(raw_data)    
    batch_len = data_len // batch_size    
    data = np.reshape(raw_data[0 : batch_size * batch_len, :], [batch_size, batch_len, -1])
    data = np.transpose(data, (1,0,2))
    
    data_length = np.reshape(raw_data_len[0 : batch_size * batch_len], [batch_size, batch_len])
    data_length = np.transpose(data_length, (1,0))
    return data, data_length 

In [0]:
ind_small_txt = 0
        
en_file = "./small_txt/" + str(ind_small_txt) + "_en.txt"
fr_file = "./small_txt/" + str(ind_small_txt) + "_fr.txt"
        
print(en_file)
print(fr_file)
        
en_input, en_input_len = generate_input_en(en_file, en_word_to_id, max_length)
fr_output, fr_output_len = generate_output_fr(fr_file, fr_word_to_id, max_length)

en_input_batches, en_input_len_batches = batch_producer(en_input, en_input_len, batch_size) 
fr_output_batches, fr_output_len_batches = batch_producer(fr_output, fr_output_len, batch_size)   

### Load test data

In [0]:
file_en_test = "./translation_test/newstest2008.en"

file_fr_test = "./translation_test/newstest2008.fr"

en_test, en_test_len = generate_input_en(file_en_test, en_word_to_id, max_length=30)

fr_test, fr_test_len = generate_input_en(file_fr_test, fr_word_to_id, max_length=30)

file_en_test = "./translation_test/newstest2009.en"

file_fr_test = "./translation_test/newstest2009.fr"

en_test_2, en_test_len_2 = generate_input_en(file_en_test, en_word_to_id, max_length=30)

fr_test_2, fr_test_len_2 = generate_input_en(file_fr_test, fr_word_to_id, max_length=30)